<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Compute_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools@main' --upgrade --force


  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-xtxw0fkn
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-xtxw0fkn
  Resolved https://github.com/fair-research/bdbag to commit 8ee285059caa71a0789179b8d57b03010ac00e7f
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/informatics-isi-edu/eye-ai-tools (to revision main) to /tmp/pip-req-build-289s5zrc
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/eye-ai-tools /tmp/pip-req-build-289s5zrc
  Resolved https://github.com/informatics-isi-edu/eye-ai-tools to commit 84aa120aa5cf805f30a1e2ae20350a61cb36d7a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ml-utils: filename=ml_utils-0.0.1-py3-none-any.whl size=9592 sha256=5a5a20bea33da4617d3631a8bd2254bfdbb0e65ff93162b53eb29cf50bbbf7e0
  Store

In [3]:
# login to DERIVA via Globus Auth
DEBUG_MODE = True #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'
!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


2023-12-06 21:55:23,665 - INFO - No default configuration file found, attempting to create one at: /root/.deriva/config.json
2023-12-06 21:55:24,291 - INFO - NoSavedTokens: No tokens were loaded
Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=8ef15ba9-2b4a-469c-a163-7fd910c9d111&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+https%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships&state=_default&response_type=code&code_challenge=PNjiByb1olCRDU91AhzytMDujWvAE_Yjqw3lVbbHmkA&code_challenge_method=S256&access_type=online&prefill_named_grant=Login+from+deriva-client+on+bd2822ae384d+%5BLinux-6.1.58%2B-x86_64-with-glibc2.35%5D+to+hosts+%5Bdev.eye-ai.org%5D+with+requested+scopes+%5Bopenid%2C+https%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all%2C+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Av

Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [4]:
# @title Configure Grader groups
grader_list = ["Benjamin Xu", "Kendra Hong", "Kent Nguyen"] # @param
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}


In [5]:
# @title Configure Process and Diagnosis Tag
process_tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Metadata = "Test Diagnosis Generation" # @param {type:"string"}
Github_URL="" # @param {type:"string"}
Process_Tag = "" # @param {type: "string"}
Description = "" # @param {type: "string"}
Github_Checksum = "" # @param {type: "string"}



Diagnosis_Tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Description = "Generate new diagnosis based on a group of graders: "+str(grader_list) # @param {type:"string"}
synonyms = [] # @param


In [6]:
from ml_utils import EyeAI
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_number = 'eye-ai')

In [7]:
# @title Define Diagnosis Generation Rule
def CDR_func(values):
    return values.mean()

def Diag_func(values):
    # Use the mode function to get the most frequent diagnosis value
    mode_diagnosis = values.mode()

    # If there are multiple modes, take the first one
    if not mode_diagnosis.empty:
        return mode_diagnosis.iloc[0]
    else:
        # Handle the case where there is no mode (e.g., all values are unique)
        return 'Unknown'

def ImageQuality_func(values):
    mode_quality = values.mode()
    if not mode_quality.empty:
        return mode_quality.iloc[0]
    else:
        return "Unknown"

In [8]:
# @title Extract the diagnosis in the catalog and generate new Diagnosis
all_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
result = EA.compute_diagnosis(all_diag[all_diag["Full_Name"].isin(grader_list)],
                                 Diag_func,
                                 CDR_func,
                                 ImageQuality_func)

In [10]:
# @title Insert New Process_Tag, Proecess, and Diagnosis_Tag
new_process_tag_RID = EA.add_term('Process_Tag', name=process_tag_name, description=grader_list, synonyms=[], exist_ok=False)
Process_RID = EA.add_process(Metadata, Github_URL, new_process_tag_RID, Description, Github_Checksum, exist_ok=False)

new_diag_tag_RID = EA.add_term('Diagnosis_Tag', Diagnosis_Tag_name, Description, exist_ok=False)


# New Section

In [11]:
entities = EA.insert_new_diagnosis(result, new_diag_tag_RID, Process_RID)